In [1]:
#We need these in this file:
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import numpy as np # linear algebra
import get_lodging_scores
import SVM_classifier_general as svm_general
import pickle
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
import glob

import sys
sys.path.append('cropping')
import load_read_name_extractor as lrne


In [2]:
data_paths_auto = glob.glob("Features/*npy")
size = int(len(data_paths_auto)/2)
features_paths = data_paths_auto[0:size]
names_paths = data_paths_auto[size:len(data_paths_auto)]

In [3]:
features_paths


['Features\\hog_features_(64, 128, 3)_cells_(10, 10)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(12, 12)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(14, 14)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(16, 16)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\hog_features_(72, 248, 3)_cells_(8, 8)_block

In [4]:
names_paths

['Features\\img_names_(64, 128, 3)_cells_(10, 10)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(12, 12)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(14, 14)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(16, 16)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(5, 5)_norm_L2.npy',
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(6, 6)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(1, 1)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(2, 2)_norm_L2.npy',
 'Features\\img_names_(72, 248, 3)_cells_(8, 8)_block_(3, 3)_norm_L2.npy',
 'Features\\img_n

In [5]:
features_paths = [
'Features\\hog_features_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy']

In [6]:
names_paths = [
 'Features\\img_names_(64, 128, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy']

In [7]:
features_load = np.load(features_paths[0], allow_pickle=True)
features_load[1000].shape

(3780,)

In [8]:
features_load[3000].shape

(3780,)

In [9]:
def save_results(results, path):
    path = path.replace("Features\\", "")
    path = "results/" + path.replace(".npy", ".txt")
    with open(path, 'a') as f:
        f.write(results)
        f.write("\n")

In [10]:
def load_labels(bin_size):
    labels = get_lodging_scores.get_labels(bin_size)
    #COnverts into set (no dublicates in set)
    my_set = set(map(tuple, labels))
    # convert set back to list of arrays
    labels = list(map(np.array, my_set))
    return labels

In [11]:
def partition_data(data_frame, percentage):
    partition = int(len(data_frame)*percentage/100)
    print(len(data_frame.shape), 2)
    print(len(data_frame[0].shape), 2)
    x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
    y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
    return x_train, x_test, y_train, y_test

In [12]:
def save_model(filename, clf):
    filename = filename.replace("Features\\", "")
    filename = "model/" + filename.replace(".npy", ".sav")
    pickle.dump(clf, open(filename, 'wb'))

In [13]:
def run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf", class_weights = None):
    for i in range (len(features_paths)):
        #laod features and names
        features_load = np.load(features_paths[i], allow_pickle=True)
        names_load = np.load(names_paths[i], allow_pickle=True)

        print(1)
        #load labels
        labels = load_labels(bin_size)

        #convert names into [flight_folder, ROI]
        names = lrne.it_name_extract_labels_from_img_jpeg(names_load)
        names_load = None
        print(2)
        #match labels with feature names
        data_frame = svm_general.match_pic_label_to_names_old(features_load, labels, names)
        print(3)
        features_load = None
        labels = None
        names = None
        print(data_frame)
        #shuffle data and convert to array
        np.random.shuffle(data_frame)

        #partition data
        percentage = 80
        data_frame = np.array(data_frame)
        x_train, x_test, y_train, y_test = partition_data(data_frame, percentage)
        data_frame = None
        features_aar = None

        #create svm
        clf = svm.SVC(kernel=svm_kernel, class_weight=class_weights)
        #fit model with training data
        clf.fit(x_train,y_train)
        #predict data
        y_pred = clf.predict(x_test)

        #Save the found model
        n_bins = int(np.floor(100/bin_size))
        print(features_paths[i])
        print(len(features_paths))
        path = "SVMC_" + "Kernel_" + svm_kernel + "_Nbins_" + str(n_bins) + "_ClassWeights_" + str(class_weights) + "_" + features_paths[i]
        save_model(path, clf)

        #Save the results
        results = "Accuracy: "+str(accuracy_score(y_test, y_pred)) + "\n" + classification_report(y_test, y_pred)
        save_results(results, path)

In [14]:
run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf")

KeyboardInterrupt: 

In [ ]:
print(features_paths[0])
print(len(features_paths))

Features\hog_features_(96, 192, 3)_cells_(8, 8)_block_(4, 4)_norm_L2.npy
1
